In [1]:
import pandas as pd
import numpy as np
import mysql.connector as msql
from mysql.connector import Error
from datetime import datetime, timedelta

In [2]:
anime = pd.read_csv('anime.tsv', delimiter = '\t')
anime.head()

,MAL_ID,Name,Score,Genders,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.77,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,254070,202552,148174,70156,23124,9938,3564,1493,797,1736
1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,32449,53084,54113,24655,6251,2023,619,232,117,397
2,6,Trigun,8.23,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,52929,80083,92549,53172,16600,6247,2097,726,344,545
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2233,4903,10341,11939,5918,2998,1110,363,169,134
4,8,Bouken Ou Beet,6.97,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,316,537,1257,1759,1104,659,266,84,50,35


In [3]:
#Cleaning
anime['Episodes'] = anime['Episodes'].replace('Unknown', 0)
anime['Score'] = anime['Score'].replace('Unknown', 0)
anime['Genders'] = anime['Genders'].replace(np.nan, 'N/A')
anime = anime.rename(columns = {"Genders": "Genres", "Premiered":"Season"})
anime['Aired'] = anime['Aired'].str.split(" to ").str[0]

In [4]:
#Convert to Datetime.
aired = []
for i, j in enumerate(anime['Aired']):
    if(anime['Aired'].str[:][i] == "Unknown"):
        aired.append("Unknown")
    else: 
        try:
            aired.append(datetime.strptime(anime['Aired'].str[:][i], '%b %d, %Y'))
        except ValueError:
            try: 
                aired.append(datetime.strptime(anime['Aired'].str[:][i], '%b %Y'))
            except ValueError:
                try: 
                    aired.append(datetime.strptime(anime['Aired'].str[:][i], '%b, %Y'))
                except ValueError:
                    aired.append(datetime.strptime(anime['Aired'].str[:][i], '%Y'))
anime['Aired'] = aired

In [5]:
#Season
season = []
for i, j in enumerate(anime['Aired']):
    if(anime['Aired'][i] == "Unknown"):
        season.append("Unknown")
    else:
        def getSeason(month):
            switcher = {
                1: "Winter",
                2: "Winter",
                3: "Winter",
                4: "Spring",
                5: "Spring", 
                6: "Spring",
                7: "Summer",
                8: "Summer",
                9: "Summer",
                10: "Fall",
                11: "Fall",
                12: "Fall"
            }
            return switcher.get(month, "Unknown")
        temp = getSeason(anime['Aired'][i].month)
        if(temp == "Unknown"):
            season.append("Unknown")
        else:
            season.append(temp + " " + str(anime['Aired'][i].year))
anime['Season'] = season

In [6]:
#Duration
duration = []
for i, j in enumerate(anime['Duration']):
    if(anime['Duration'].str[:][i] == "Unknown"):
        duration.append("Unknown")
    else:
        try: 
            duration.append(datetime.strptime(anime['Duration'].str[:][i], '%M min.'))
        except ValueError:
            try: 
                duration.append(datetime.strptime(anime['Duration'].str[:][i], '%M min. per ep.'))
            except ValueError:
                try: 
                    duration.append(datetime.strptime(anime['Duration'].str[:][i], '%H hr.'))
                except ValueError:
                    try: 
                        duration.append(datetime.strptime(anime['Duration'].str[:][i], '%H hr. per ep.'))
                    except ValueError:
                        try: 
                            duration.append(datetime.strptime(anime['Duration'].str[:][i], '%H hr. %M min.'))
                        except ValueError:
                            try: 
                                duration.append(datetime.strptime(anime['Duration'].str[:][i], '%H hr. %M min. per ep.'))
                            except ValueError:
                                try: 
                                    duration.append(datetime.strptime(anime['Duration'].str[:][i], '%S sec.'))
                                except ValueError:
                                    duration.append(datetime.strptime(anime['Duration'].str[:][i], '%S sec. per ep.'))

In [7]:
total_duration = []
def convertTimeDelta(d):
    if(d == "Unknown"):
        return "Unknown"
    else:
        return timedelta(hours = d.hour, minutes = d.minute, seconds = d.second)
temp = [convertTimeDelta(x) for x in duration]
anime['Duration'] = temp
for i, j in enumerate(anime['Episodes']):
    if(temp[i] == "Unknown"):
        total_duration.append("Unknown")
    else:
        total_duration.append(temp[i] * int(anime['Episodes'][i]))
anime['Total Duration'] = total_duration

In [8]:
#1nf
multi_columns = ['Genres', 'Producers', 'Licensors', 'Studios']
for i in multi_columns:
    anime[i] = anime[i].str.split(", ")

In [9]:
anime_1nf = pd.DataFrame()
for i, j in enumerate(anime['MAL_ID']):
    product = 1
    for k in multi_columns:
        product *= len(anime[k].iloc[i])
    temp = pd.DataFrame([anime.iloc[i]] * product)
    genres = []
    producers = []
    licensors = []
    studios = []
    genre_length = 0
    producer_length = 0
    licensor_length = 0
    studio_length = 0
    for product_length in range(product):
        genres.append(temp['Genres'].iloc[0][genre_length])
        producers.append(temp['Producers'].iloc[0][producer_length])
        licensors.append(temp['Licensors'].iloc[0][licensor_length])
        studios.append(temp['Studios'].iloc[0][studio_length])
        if(genre_length == len(temp['Genres'].iloc[0]) - 1):
            if(producer_length != len(temp['Producers'].iloc[0]) - 1):
                genre_length = 0
                producer_length += 1
            elif(licensor_length != len(temp['Licensors'].iloc[0]) - 1):
                genre_length = 0
                producer_length = 0
                licensor_length += 1
            elif(studio_length != len(temp['Studios'].iloc[0]) - 1):
                genre_length = 0
                producer_length = 0
                licensor_length = 0
                studio_length += 1
        else:
            genre_length += 1
    temp['Genres'] = genres
    temp['Producers'] = producers
    temp['Licensors'] = licensors
    temp['Studios'] = studios
    print(f"\r{i}/{len(anime) - 1}", end="")
    anime_1nf = anime_1nf.append(temp)

18321/18321

In [10]:
anime_1nf

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Season,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Total Duration
0,1,Cowboy Bebop,8.77,Action,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,Spring 1998,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
0,1,Cowboy Bebop,8.77,Adventure,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,Spring 1998,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
0,1,Cowboy Bebop,8.77,Comedy,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,Spring 1998,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
0,1,Cowboy Bebop,8.77,Drama,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,Spring 1998,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
0,1,Cowboy Bebop,8.77,Sci-Fi,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,Spring 1998,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18320,49471,Argonavis Movie: Ryuusei no Obligato,0,Music,Argonavis the Movie:Ryusei no Obligato,劇場版アルゴナビス 流星のオブリガート,Movie,1,2021-11-19 00:00:00,Fall 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
18320,49471,Argonavis Movie: Ryuusei no Obligato,0,Drama,Argonavis the Movie:Ryusei no Obligato,劇場版アルゴナビス 流星のオブリガート,Movie,1,2021-11-19 00:00:00,Fall 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
18320,49471,Argonavis Movie: Ryuusei no Obligato,0,Music,Argonavis the Movie:Ryusei no Obligato,劇場版アルゴナビス 流星のオブリガート,Movie,1,2021-11-19 00:00:00,Fall 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
18320,49471,Argonavis Movie: Ryuusei no Obligato,0,Drama,Argonavis the Movie:Ryusei no Obligato,劇場版アルゴナビス 流星のオブリガート,Movie,1,2021-11-19 00:00:00,Fall 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [55]:
duration = []
total_duration = []
anime_1nf = anime_1nf.reset_index()
for i, j in enumerate(anime_1nf['Duration']):
    duration.append(str(anime_1nf['Duration'][i]))
    total_duration.append(str(anime_1nf['Total Duration'][i]))
anime_1nf['Duration'] = duration
anime_1nf['Total Duration'] = total_duration
anime_1nf = anime_1nf.drop(columns = ['index'])
anime_1nf.head()

,index,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,Total Duration
0,0,1,Cowboy Bebop,8.77,Action,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
1,0,1,Cowboy Bebop,8.77,Adventure,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
2,0,1,Cowboy Bebop,8.77,Comedy,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
3,0,1,Cowboy Bebop,8.77,Drama,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00
4,0,1,Cowboy Bebop,8.77,Sci-Fi,Cowboy Bebop,カウボーイビバップ,TV,26,1998-04-03 00:00:00,...,202552,148174,70156,23124,9938,3564,1493,797,1736,0 days 10:24:00


In [59]:
try:
    #conn = msql.connect(host='REDACTED', user='REDACTED', password='REDACTED', database = 'REDACTED')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute('DROP TABLE IF EXISTS anime;')
        cursor.execute('CREATE TABLE anime(`MAL_ID` DOUBLE,`Name` VARCHAR(100),`Score` DOUBLE,`Genres` VARCHAR(200),`English_name` VARCHAR(200),`Japanese_name` VARCHAR(100),`Type` VARCHAR(100),`Episodes` DOUBLE,`Aired` VARCHAR(100),`Season` VARCHAR(100),`Producers` VARCHAR(400),`Licensors` VARCHAR(100),`Studios` VARCHAR(200),`Source` VARCHAR(100),`Duration` VARCHAR(400),`Rating` VARCHAR(100),`Ranked` VARCHAR(100),`Popularity` VARCHAR(100),`Members` VARCHAR(100),`Favorites` VARCHAR(100),`Watching` VARCHAR(100),`Completed` VARCHAR(100),`OnHold` VARCHAR(100),`Dropped` VARCHAR(100),`Plan_to_Watch` VARCHAR(100),`Score10` VARCHAR(100),`Score9` VARCHAR(100),`Score8` VARCHAR(100),`Score7` VARCHAR(100),`Score6` VARCHAR(100),`Score5` VARCHAR(100),`Score4` VARCHAR(100),`Score3` VARCHAR(100),`Score2` VARCHAR(100),`Score1` VARCHAR(100),`Total Duration` VARCHAR(400));')
        for i,row in anime_1nf.iterrows():
            sql = "INSERT INTO anime VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s)"
            cursor.execute(sql, tuple(row))
            conn.commit()
except Error as e:
    print("Error while connecting to MySQL", e)